# Real Market Data Backtesting with q/KDB-X

This notebook demonstrates how to build a **financial backtesting system** using KDB-X/q. We'll simulate a simple trading strategy and calculate how much profit or loss it would have generated.

## What we explore in this Notebook?


- **Real market data** from Yahoo Finance (last 8 days, minute-level)
- **Moving average crossover strategy** - a classic momentum approach
- **15-minute bars** - aggregated from minute data for cleaner signals
- **Complete trade analysis** - PnL, win rate, drawdowns, and more

## The Strategy: Moving Average Crossover

We'll implement a **dual moving average crossover**:
- **Fast MA**: 10-period moving average (responsive to recent price action)
- **Slow MA**: 30-period moving average (tracks longer-term trend)
- **Buy signal**: When fast MA crosses *above* slow MA → bullish momentum
- **Sell signal**: When fast MA crosses *below* slow MA → bearish momentum

This is a trend-following strategy used by traders to ride momentum waves.

## Prerequisites

- Requires KDB-X to be installed, you can sign up and download on [Developer Center](https://developer.kx.com/products/kdb-x/install). For full install instructions see: [KDB-X Install](https://code.kx.com/kdb-x/).

- To Install [KDB-X Python](https://code.kx.com/pykx/4.0/examples/jupyter-integration.html): `pip install --upgrade --pre pykx` (This is required to run q in a Python notebook, but you can run the q directly from a q session if you prefer.)



In [3]:
!pip install -qq --upgrade --pre pykx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 72.2 MB/s eta 0:00:00


In [ ]:
import pykx as kx
import yfinance as yf
from datetime import datetime, timedelta

## Step 1: Fetch Real Market Data

We'll download the last 8 days of minute-level data for **AAPL** and **MSFT**. Yahoo Finance provides free intraday data, though it's limited to the most recent period.

### Why minute data?
Minute bars give us fine-grained price action, allowing us to:
- Aggregate into any timeframe we want (5min, 15min, 1hr)
- Simulate realistic entry/exit timing
- See intraday volatility patterns

We'll use Python/yfinance to fetch the data, then convert it to a q table for all subsequent analysis.

In [5]:
# Calculate dynamic date range (last 8 days)
end_date = datetime.now()
start_date = end_date - timedelta(days=8)

print(f"Fetching data from {start_date.date()} to {end_date.date()}...")

# Download minute-level data
data = yf.download(
    tickers=["AAPL", "MSFT"],
    start=start_date.strftime("%Y-%m-%d"),
    end=end_date.strftime("%Y-%m-%d"),
    interval="1m",
    auto_adjust=True
)

# Reshape from wide (columns per ticker) to long format (rows per ticker)
data_long = data.stack(level=1).reset_index()

# Rename columns to match q conventions
data_long = data_long.rename(columns={
    'Datetime': 'dt',
    'Ticker': 'sym'
})

# Convert ticker symbols to lowercase (q convention)
data_long['sym'] = data_long['sym'].str.lower()

# Convert to q table - this is our bridge from Python to q
quotes = kx.toq(data_long)
kx.q['quotes'] = quotes

print(f"✓ Loaded {len(data_long):,} minute bars")
print(f"✓ Tickers: {', '.join(data_long['sym'].unique())}")

Fetching data from 2026-01-20 to 2026-01-28...


[*********************100%***********************]  2 of 2 completed

✓ Loaded 4,664 minute bars
✓ Tickers: aapl, msft



/tmp/ipython-input-3175797431.py:17: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data_long = data.stack(level=1).reset_index()


## Step 2: Inspect the Data Structure

Let's examine what we're working with. From here on, we use **pure q code** - no more Python!

The data should have columns: `dt` (datetime), `sym` (symbol), `Open`, `High`, `Low`, `Close`, `Volume`.

In [6]:
# Enable q-first mode - write pure q code in subsequent cells
kx.util.jupyter_qfirst_enable()

KDB-X Python now running in 'jupyter_qfirst' mode. All cells by default will be run as q code.
Include '%%py' at the beginning of each cell to run as python code. 


In [7]:
/ Adjust console width for visibility
\c 20 200

In [9]:
/ Show first 10 rows
10 sublist quotes

dt                            sym  Close   High    Low     Open   Volume 
-------------------------------------------------------------------------
2026.01.20D14:30:00.000000000 aapl 254.48  254.79  252.36  252.51 2600373
2026.01.20D14:30:00.000000000 msft 450.45  452.13  450.1   451.43 1289307
2026.01.20D14:31:00.000000000 aapl 253.9   254.64  253.809 254.39 250428 
2026.01.20D14:31:00.000000000 msft 450.12  450.86  449.33  450.51 253023 
2026.01.20D14:32:00.000000000 aapl 254.3   254.57  253.83  253.91 232262 
2026.01.20D14:32:00.000000000 msft 449.74  450.175 449.3   450.06 99677  
2026.01.20D14:33:00.000000000 aapl 254.38  254.79  254.19  254.27 192391 
2026.01.20D14:33:00.000000000 msft 450.32  450.5   449.28  449.69 89392  
2026.01.20D14:34:00.000000000 aapl 254.07  254.39  253.85  254.37 181225 
2026.01.20D14:34:00.000000000 msft 451.395 451.5   450.03  450.5  81703  


In [10]:
/ Get data summary by symbol
select
  bars:count i,
  start:min dt,
  end:max dt,
  low_price:min Close,
  high_price:max Close
  by sym from quotes



sym | bars start                         end                           low_price high_price
----| -------------------------------------------------------------------------------------
aapl| 2339 2026.01.20D14:30:00.000000000 2026.01.27D20:59:00.000000000 243.44    261.88    
msft| 2325 2026.01.20D14:30:00.000000000 2026.01.27D20:59:00.000000000 438.8001  482.71    


## Step 3: Aggregate to 15-Minute Bars

Minute data can be noisy. We'll aggregate it into **15-minute OHLCV bars** for cleaner signals.

### The Process:
1. Extract the time component from each datetime
2. Use [`xbar`](https://code.kx.com/q/ref/xbar/) to bucket times into 15-minute intervals
3. Aggregate: first Open, max High, min Low, last Close, sum Volume
4. Group by symbol and time bucket

This is standard OHLC bar construction used throughout finance.

In [11]:
/ Extract date and time components
quotes:update date:`date$dt, time:`time$dt from quotes;

/ Create 15-minute time buckets using xbar
/ xbar rounds down to nearest interval (e.g., 14:37 -> 14:30)
quotes:update bucket:15 xbar time from quotes;

/ Aggregate to 15-minute bars
bars:select
  open:first Open,
  high:max High,
  low:min Low,
  close:last Close,
  volume:sum Volume
  by sym, date, bucket from quotes;

/ Recreate datetime column and clean up
bars:update dt:date+bucket from bars;
bars:delete date,bucket from bars;

/ Sort by symbol and datetime for efficiency
bars:`sym`dt xasc bars;

"15-minute bars created:"
10 sublist bars

15-minute bars created:
sym  open   high   low     close   volume  dt                           
------------------------------------------------------------------------
aapl 252.51 254.79 252.36  254.48  2600373 2026.01.20D14:30:00.000000000
aapl 254.39 254.64 253.809 253.9   250428  2026.01.20D14:31:00.000000000
aapl 253.91 254.57 253.83  254.3   232262  2026.01.20D14:32:00.000000000
aapl 254.27 254.79 254.19  254.38  192391  2026.01.20D14:33:00.000000000
aapl 254.37 254.39 253.85  254.07  181225  2026.01.20D14:34:00.000000000
aapl 254.02 254.16 253.3   253.57  186460  2026.01.20D14:35:00.000000000
aapl 253.55 253.58 252.5   252.76  152844  2026.01.20D14:36:00.000000000
aapl 252.76 252.99 252.57  252.67  121456  2026.01.20D14:37:00.000000000
aapl 252.64 253.66 252.47  253.465 142247  2026.01.20D14:38:00.000000000
aapl 253.43 253.44 253.03  253.08  124974  2026.01.20D14:39:00.000000000


## Step 4: Calculate Moving Averages

Now we calculate our strategy indicators:
- **Fast MA**: 10-period moving average using [`mavg`](https://code.kx.com/q/ref/avg/#mavg)
- **Slow MA**: 30-period moving average

The `by sym` clause ensures we calculate MAs separately for each ticker - no data leakage between symbols!

A 10-period moving average is the average closing price of the last 10 fifteen-minute bars (fast-moving, reacts quickly to recent price changes), while a 30-period moving average is the average closing price of the last 30 fifteen-minute bars (slow-moving, smooths out short-term fluctuations to show the longer-term trend).

### Identifying Crossovers
A crossover occurs when:
- **Bullish**: Fast MA was ≤ Slow MA, now Fast MA > Slow MA
- **Bearish**: Fast MA was ≥ Slow MA, now Fast MA < Slow MA

We use [`prev`](https://code.kx.com/q/ref/next/) to access the previous row's values for comparison.

In [12]:
/ Calculate moving averages per symbol
bars:update
  ma_fast:mavg[10;close],
  ma_slow:mavg[30;close]
  by sym from bars;

/ Get previous values for crossover detection
bars:update
  prev_fast:prev ma_fast,
  prev_slow:prev ma_slow
  by sym from bars;

/ Identify crossover signals
/ signal = 1: bullish cross (BUY)
/ signal = -1: bearish cross (SELL)
/ signal = 0: no signal
bars:update signal:0 from bars;
bars:update signal:1 from bars where (ma_fast>ma_slow) and prev_fast<=prev_slow;
bars:update signal:-1 from bars where (ma_fast<ma_slow) and prev_fast>=prev_slow;

"Trading signals generated:"
select from bars where signal<>0

Trading signals generated:
sym  open     high   low      close    volume  dt                            ma_fast  ma_slow  prev_fast prev_slow signal
-------------------------------------------------------------------------------------------------------------------------
aapl 253.09   253.14 252.6    252.74   123556  2026.01.20D14:40:00.000000000 253.4935 253.5832 253.6675  253.6675  -1    
aapl 251.4701 251.74 251.445  251.6157 99923   2026.01.20D15:29:00.000000000 251.5011 251.4717 251.4335  251.5038  1     
aapl 251.29   251.33 251.19   251.215  54891   2026.01.20D15:39:00.000000000 251.3346 251.3358 251.3747  251.3416  -1    
aapl 251.78   252.06 251.76   252.06   79390   2026.01.20D16:01:00.000000000 251.338  251.3233 251.244   251.3     1     
aapl 251.45   251.55 251.36   251.36   42067   2026.01.20D16:21:00.000000000 251.545  251.5538 251.592   251.5458  -1    
aapl 251.5084 251.55 251.43   251.445  48272   2026.01.20D16:40:00.000000000 251.3716 251.3599 251.3331  251.3725  1   

## Step 5: Generate Trade Pairs

We have signals, now we need to pair them up into actual trades:
- Each **BUY signal** (signal=1) is an entry
- Each **SELL signal** (signal=-1) is an exit
- We match each entry with the next exit using an asof join

### Why Asof Join?
The [`aj`](https://code.kx.com/q/ref/aj/) function finds the "most recent match as of a time" - perfect for matching "next exit after this entry".

We'll trade 100 shares per position for simplicity.

In [13]:
/ Extract buy and sell signals
trades:select sym, dt, signal, close from bars where signal<>0;
trades:update tradetype:`buy from trades where signal=1;
trades:update tradetype:`sell from trades where signal=-1;

/ Separate buys and sells
buys:select sym, entry_dt:dt, entry_price:close from trades where tradetype=`buy;
sells:select sym, exit_dt:dt, exit_price:close from trades where tradetype=`sell;

/ Apply sorted attribute for efficient joining
buys:update `s#sym from `sym xasc buys;
sells:update `s#sym from `sym xasc sells;

/ Match each SELL with its most recent preceding BUY using asof join (backward-looking)
/ Rename entry_dt→exit_dt temporarily so aj finds the buy that occurred at or before each sell
positions:aj[`sym`exit_dt;sells;update exit_dt:entry_dt from buys];
positions:select sym, entry_dt, entry_price, exit_dt, exit_price from positions;

/ Remove incomplete trades where no matching buy was found (null entry_price)
positions:delete from positions where null entry_price;

/ Set position size
positions:update size:100 from positions;

"Generated positions:"
positions

Generated positions:
sym  entry_dt                      entry_price exit_dt                       exit_price size
--------------------------------------------------------------------------------------------
aapl 2026.01.20D15:29:00.000000000 251.6157    2026.01.20D15:39:00.000000000 251.215    100 
aapl 2026.01.20D16:01:00.000000000 252.06      2026.01.20D16:21:00.000000000 251.36     100 
aapl 2026.01.20D16:40:00.000000000 251.445     2026.01.20D16:44:00.000000000 250.8503   100 
aapl 2026.01.20D17:03:00.000000000 251.15      2026.01.20D17:14:00.000000000 250.76     100 
aapl 2026.01.20D19:44:00.000000000 247.703     2026.01.20D19:51:00.000000000 247.105    100 
aapl 2026.01.20D20:59:00.000000000 246.69      2026.01.21D15:07:00.000000000 246.4138   100 
aapl 2026.01.21D15:20:00.000000000 247.165     2026.01.21D16:02:00.000000000 247.92     100 
aapl 2026.01.21D17:25:00.000000000 246.0199    2026.01.21D18:05:00.000000000 245.83     100 
aapl 2026.01.21D18:14:00.000000000 246.2       20

## Step 6: Calculate PnL

For each position, we calculate:
- **PnL in dollars**: `size × (exit_price - entry_price)`
- **PnL percentage**: `100 × (exit_price - entry_price) / entry_price`
- **Holding period**: How long we held the position

This tells us which trades were winners and losers.

In [14]:
/ Calculate PnL metrics for each trade
positions:update
  pnl:size * exit_price - entry_price,
  pnl_pct:100 * (exit_price - entry_price) % entry_price,
  holding_period:exit_dt - entry_dt
  from positions;

"Position PnL: "
positions

Position PnL: 
sym  entry_dt                      entry_price exit_dt                       exit_price size pnl       pnl_pct     holding_period      
---------------------------------------------------------------------------------------------------------------------------------------
aapl 2026.01.20D15:29:00.000000000 251.6157    2026.01.20D15:39:00.000000000 251.215    100  -40.07111 -0.1592552  0D00:10:00.000000000
aapl 2026.01.20D16:01:00.000000000 252.06      2026.01.20D16:21:00.000000000 251.36     100  -69.99969 -0.2777104  0D00:20:00.000000000
aapl 2026.01.20D16:40:00.000000000 251.445     2026.01.20D16:44:00.000000000 250.8503   100  -59.47113 -0.2365174  0D00:04:00.000000000
aapl 2026.01.20D17:03:00.000000000 251.15      2026.01.20D17:14:00.000000000 250.76     100  -38.99994 -0.1552854  0D00:11:00.000000000
aapl 2026.01.20D19:44:00.000000000 247.703     2026.01.20D19:51:00.000000000 247.105    100  -59.80072 -0.2414211  0D00:07:00.000000000
aapl 2026.01.20D20:59:00.00000000

In [15]:
/ Extract the pnl values
pnl:positions`pnl;

## Step 7: Performance by Symbol

Different stocks behave differently. Let's break down performance by ticker to see which one the strategy works better on.

We might find:
- One stock trends more consistently (better for MA crossover)
- One stock is choppier (more whipsaws, lower win rate)
- Differences in volatility and trade frequency

In [18]:
/ Aggregate statistics by symbol
symbol_stats:select
  trades:count i,
  winners:sum pnl>0,
  total_pnl:sum pnl,
  avg_pnl:avg pnl,
  win_rate:100 * (sum pnl>0) % count i,
  best_trade:max pnl,
  worst_trade:min pnl
  by sym from positions;

-1 "\n=== PERFORMANCE BY SYMBOL ===";
symbol_stats

sym | trades winners total_pnl avg_pnl  win_rate best_trade worst_trade
----| -----------------------------------------------------------------
aapl| 48     14      251.4755  5.239073 29.16667 405.0003   -94.00024  
msft| 45     15      2006.519  44.5893  33.33333 1609       -162.9974  


## Step 8: Cumulative PnL Over Time

To understand the strategy's progression, we calculate cumulative PnL - how our account balance would have grown (or shrunk) over the backtest period.

The [`sums`](https://code.kx.com/q/ref/sum/#sums) function creates a running total, perfect for equity curves.

In [19]:
/ Sort trades chronologically and calculate running PnL
pnl_timeline:`entry_dt xasc select entry_dt, sym, pnl from positions;
pnl_timeline:update cumulative_pnl:sums pnl from pnl_timeline;

"CUMULATIVE PnL OVER TIME: "
-15 sublist pnl_timeline

CUMULATIVE PnL OVER TIME: 
entry_dt                      sym  pnl       cumulative_pnl
-----------------------------------------------------------
2026.01.27D17:13:00.000000000 aapl -19.00024 2538.133      
2026.01.27D17:15:00.000000000 aapl -9.960938 2528.172      
2026.01.27D17:30:00.000000000 msft 39.4989   2567.671      
2026.01.27D18:13:00.000000000 aapl -27.00195 2540.669      
2026.01.27D18:14:00.000000000 msft -16.99829 2523.671      
2026.01.27D18:47:00.000000000 msft 26.32141  2549.992      
2026.01.27D19:05:00.000000000 aapl -13.50098 2536.491      
2026.01.27D19:21:00.000000000 aapl -22.50061 2513.991      
2026.01.27D19:29:00.000000000 msft -42.99927 2470.992      
2026.01.27D19:33:00.000000000 msft -6.500244 2464.491      
2026.01.27D19:56:00.000000000 msft -46.49963 2417.992      
2026.01.27D19:59:00.000000000 aapl -20.49866 2397.493      
2026.01.27D20:42:00.000000000 msft -29.49829 2367.995      
2026.01.27D20:47:00.000000000 aapl -55.00183 2312.993      
2026.01.27D20

## Step 9: Maximum Drawdown Analysis

**Maximum drawdown** is the largest peak-to-trough decline in cumulative PnL. It measures the strategy's worst-case risk.

### Why it matters:
- Shows how much capital you could have lost at the worst point
- Helps determine position sizing and risk tolerance
- More realistic than just looking at final PnL

We calculate:
1. Running maximum PnL at each point (the peak)
2. Current drawdown = Current PnL - Running maximum
3. Maximum drawdown = Largest negative value

In [20]:
/ Calculate running maximum (the peak)
pnl_timeline:update running_max:maxs cumulative_pnl from pnl_timeline

/ Calculate drawdown from peak
pnl_timeline:update drawdown:cumulative_pnl - running_max from pnl_timeline

/ Find maximum drawdown
max_dd:min pnl_timeline[`drawdown]
"Maximum Drawdown: $", string max_dd

/ Show when it occurred
"Worst drawdown period:"
select from pnl_timeline where drawdown = min drawdown

Maximum Drawdown: $-747.67
Worst drawdown period:
entry_dt                      sym  pnl       cumulative_pnl running_max drawdown
--------------------------------------------------------------------------------
2026.01.21D18:55:00.000000000 msft -150.5005 -478.4195      269.2505    -747.67 


## Step 10: Testing Different Parameters

What if we used **different MA periods**? Let's try a faster 5/20 combination and compare signal frequency.

### Trade-offs:
- **Shorter MAs** (5/20): More signals, more responsive, but more whipsaws
- **Longer MAs** (10/30): Fewer signals, smoother, but slower to react

This is called **parameter sensitivity analysis** - testing how results change with different settings.

In [21]:
/ Calculate alternative moving averages (5/20)
bars2:update
  ma_fast5:mavg[5;close],
  ma_slow20:mavg[20;close]
  by sym from bars;

/ Detect crossovers with new parameters
/ To detect a "crossover," we need to know the values from the PREVIOUS row.
/ 'prev' shifts the data down by one, allowing us to compare 'now' vs 'then'.
bars2:update
  prev_fast5:prev ma_fast5,
  prev_slow20:prev ma_slow20
  by sym from bars2;

/ Initialize all signals to 0 (No Trade/Hold)
bars2:update signal2:0 from bars2;

/ BULLISH CROSSOVER:
/ Fast MA is currently ABOVE slow MA, but was BELOW or EQUAL in the previous bar.
bars2:update signal2:1 from bars2 where (ma_fast5>ma_slow20) and prev_fast5<=prev_slow20;

/ BEARISH CROSSOVER:
/ Fast MA is currently BELOW slow MA, but was ABOVE or EQUAL in the previous bar
bars2:update signal2:-1 from bars2 where (ma_fast5<ma_slow20) and prev_fast5>=prev_slow20;

/ Compare signal counts
signal_count:([]
  strategy:`$("MA_10_30";"MA_5_20");
  signals:(sum abs bars[`signal]; sum abs bars2[`signal2])
  );

"SIGNAL FREQUENCY COMPARISON: "
signal_count

SIGNAL FREQUENCY COMPARISON: 
strategy signals
----------------
MA_10_30 187    
MA_5_20  285    


#### Challenge: Can you find the PnL stats for this new 5/20 Moving Average strategy?


In [ ]:
/ Challenge Code:

## Conclusion and Next Steps

### What We Built

This notebook demonstrated:
1. **Real data integration**: Yahoo Finance → q/KDB-X pipeline
2. **Bar aggregation**: Minute data → 15-minute OHLC bars
3. **Technical indicators**: Moving averages with `mavg`
4. **Signal generation**: Crossover detection logic
5. **Trade matching**: Entry/exit pairing with `aj`
6. **Performance analysis**: PnL, win rate, drawdown, cumulative returns
7. **Parameter testing**: Comparing different MA periods

### Key q/KDB-X Features Used

- **`xbar`**: Time bucketing for bar aggregation
- **`mavg`**: Efficient moving averages
- **`by` clause**: Per-symbol calculations without loops
- **`aj` (asof join)**: Time-series matching for trade pairing
- **`sums` / `maxs`**: Running calculations for equity curves
- **`prev`**: Access previous row values for crossover detection
- **Attributes (`` `s# ``)**: Performance optimization for sorted data

### Improvements for Production

To make this a real trading strategy, add:

1. **Transaction costs**: Commissions ($0.005/share?), slippage (1-2 bps?)
2. **Position sizing**: Risk-based sizing, not fixed 100 shares
3. **Risk management**: Stop losses, maximum position limits
4. **Multiple timeframes**: Confirm signals on higher timeframes
5. **Volume filters**: Ignore signals on low-volume bars
6. **Market regime detection**: Different parameters for trending vs ranging markets
7. **Walk-forward testing**: Test on rolling windows to avoid overfitting
8. **Realistic execution**: Model market impact, time-in-force

### Why q/KDB-X?

This example used just 8 days of minute data. In production:
- Backtests run on **years of tick data** (billions of records)
- Real-time strategies process **millions of quotes per second**
- Portfolio analytics aggregate **thousands of instruments simultaneously**

q/KDB-X excels at these scales with the same clean syntax we used here. The code patterns are identical whether you're processing 10,000 rows or 10 billion.

### Resources

- [q Reference Card](https://code.kx.com/q/ref/)
- [Time-series joins (aj, asof)](https://code.kx.com/q/ref/aj/)
- [Moving averages (mavg)](https://code.kx.com/q/ref/avg/#mavg)
- [KDB-X Python Documentation](https://code.kx.com/pykx/)

Happy backtesting! 🚀